In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split


df = pd.read_csv("./dataset/dataset.csv")

y_total = df.pop("smoking").astype(int)
X_total = df

X, X_check, y, y_check = train_test_split(
    X_total,
    y_total,
    test_size = 5000,
    stratify = y_total,
    random_state = 5510
)


In [3]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report

rf_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("rf", RandomForestClassifier(
        n_estimators=4000,
        min_samples_leaf=2,
        class_weight="balanced_subsample",
        n_jobs=-1,
        random_state=123
    ))
])

rf_pipe.fit(X,y)

proba = rf_pipe.predict_proba(X_check)[:, 1]
pred  = (proba >= 0.5).astype(int)

print(classification_report(y_check, pred, digits=3))

              precision    recall  f1-score   support

           0      0.951     0.975     0.963      2490
           1      0.974     0.950     0.962      2510

    accuracy                          0.962      5000
   macro avg      0.963     0.962     0.962      5000
weighted avg      0.963     0.962     0.962      5000



In [4]:

rf = rf_pipe.named_steps["rf"]
import pandas as pd
imp = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
print("\nTop features:\n", imp)


Top features:
 Gtp                    0.096642
triglyceride           0.080147
hemoglobin             0.068514
LDL                    0.062229
Cholesterol            0.059833
HDL                    0.059187
waist(cm)              0.058425
systolic               0.057129
fasting blood sugar    0.056501
ALT                    0.056064
AST                    0.051144
relaxation             0.050704
age                    0.042021
height(cm)             0.035691
serum creatinine       0.033758
weight(kg)             0.033222
eyesight(right)        0.033031
eyesight(left)         0.032760
gender                 0.013260
dental caries          0.011023
Urine protein          0.004846
hearing(right)         0.002012
hearing(left)          0.001854
dtype: float64


In [ ]:
# upsampling auf Female / Female + smokers

# RandomForestClassifier, Support Vector Machine, MLP (Multi Layer)

# Hyperparameter, Gridsearch

# Correlation, Signifikanztests

# Literaturrecherche (verweis auf die Correlations, etc.)

#